# Expand window

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt

from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Load Data

In [3]:
train_data = pd.read_csv("../../data/train/processed.csv")

In [4]:
print(train_data.shape)
train_data.sample(5)

(164309, 14)


,Loan_Amount_Requested,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate,Experience,Closed_Accounts
90509,8.006701,0,10.596660,2,1,1,0,65.0,2.079442,19,1,2,2.0,12
61547,8.630700,0,10.645449,2,2,4,0,31.0,2.484907,24,0,1,3.0,13
132242,9.741027,0,11.184435,0,2,4,3,78.0,3.044522,39,1,2,10.0,19
62211,9.873080,0,11.362114,0,2,3,3,49.0,2.484907,34,1,3,10.0,23
130046,9.672564,4,11.050906,0,2,1,0,31.0,2.079442,30,1,2,6.0,23


In [14]:
test_data = pd.read_csv("../../data/test/processed.csv")
print(test_data.shape)
test_data.head()


(109541, 14)


,Loan_ID,Loan_Amount_Requested,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Experience,Closed_Accounts
0,10164310,10.221978,0,11.767575,0,2,2,0,68.0,2.397895,37,1,10.0,27
1,10164311,10.165890,4,11.608245,2,1,2,0,31.0,1.945910,23,1,10.0,17
2,10164312,8.712102,4,11.225257,0,2,1,2,31.0,1.791759,20,1,0.5,15
3,10164313,9.392745,0,11.198228,1,2,1,0,31.0,1.945910,8,1,10.0,2
4,10164314,10.463132,0,11.957618,2,2,1,0,26.0,2.302585,21,1,0.5,12


# Split X and Y

In [6]:
target = 'Interest_Rate'
X = train_data.drop(target, axis = 1)
y = train_data[target]

In [7]:
X.shape

(164309, 13)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)
print("Train data size", X_train.shape[0])
print("Test data size", X_test.shape[0])

Train data size 131447
Test data size 32862


# Model Selection

In [ ]:
def get_random_forest_params_dict():
    # Random Forest Parameters
    n_estimators = [100,200,300,400] # Number of decision trees in random forest
    max_features = ['auto', 'sqrt'] # Number of features to consider at every split
    max_depth = [10,20,30,40,50,60] # Maximum number of levels in tree
    bootstrap = [True, False] # Method of selecting samples for training each tree

    rf_param_grid = dict(max_features=max_features, n_estimators=n_estimators, max_depth=max_depth, bootstrap=bootstrap)
    
    return rf_param_grid

In [ ]:
def get_decision_tree_params_dict():
    criterion = ['gini', 'entropy']
    max_depth = [1,2,3,4,6,8,12]

    deci_tree_parm_grid = dict(criterion = criterion, max_depth = max_depth)
    
    return deci_tree_parm_grid

In [ ]:
def get_knn_params_dict():
    
    knn_range = list(range(5,11))
    weight_options = ["uniform", "distance"]

    knn_param_grid = dict(n_neighbors = knn_range, weights = weight_options)

    return knn_param_grid

In [ ]:
def get_svm_params_dict():
    svm_param_grid = dict(C = [0.1, 1, 5], gamma = [1, 0.1,  0.001], kernel = ['rbf', 'poly', 'sigmoid'])

    return svm_param_grid

In [ ]:
def process_models(train_features, train_target, test_features, test_target):
    
    features_selected = list(train_features.columns.values)
    
    # Models which we are going to analyse
    base_models = {
#         'KNN': KNeighborsClassifier(),
        'DecisionTree': DecisionTreeClassifier(random_state=10),
#         'RandomForest': RandomForestClassifier(random_state=10),
#         'SVM': SVC()
    }

    # Model's parameters required for hypterparameter tuning
    params_grid = {
#         'KNN': get_knn_params_dict(),
        'DecisionTree': get_decision_tree_params_dict(),
#         'RandomForest': get_random_forest_params_dict(),
#         'SVM': get_svm_params_dict()
    }
    
    # Array to Store all models summary
    model_summary_lst = []
    
    for name in base_models.keys():

        # Base estimator or model
        estimator = base_models[name] 
        
        #================================== Scale features ============================================
        if name == 'KNN':
            # KNN performs very well when data is scaled. So we first scale the data and them fit it 
            print("Data scaled for KNN")
            scalar = StandardScaler()

            train_features = scalar.fit_transform(train_features)
            test_features = scalar.fit_transform(test_features)
        
        
        #================================== Base Model ============================================
        base_model = estimator
        
        # Fit model
        base_model.fit(train_features, train_target)
        
        #Predict test data using base model
        base_y_pred = base_model.predict(test_features)
        
        #Base model accuracy
        base_accuracy = accuracy_score(test_target, base_y_pred) 
        
        
        
        #================= Best fit Model with parameter tuning ===================================
        
        
        #RandomizedSearchCV model with parameter tuning
        est_params = params_grid[name] 
        random_search_cv = RandomizedSearchCV(estimator, est_params, n_iter=12, cv=10, verbose=0, n_jobs=-1,random_state=10)
       
        # Fit RandomizedSearchCV object
        random_search_cv.fit(train_features, train_target)
        
        #Predict test data using best fit model
        random_search_cv_y_pred = random_search_cv.predict(test_features)
        
        #Best fit accuracy
        best_fit_accuracy = accuracy_score(test_target, random_search_cv_y_pred)  
        
        #================= Create dataframe with accuracy summary details ===================================
         
        #Calculate the improvemnt of best fit model's accuracy over our base model accuracy
        improvement ='{:0.3f}%.'.format( 100 * (best_fit_accuracy - base_accuracy) / base_accuracy)

        #Add summary to object
        model_summary_lst.append([name, train_features.shape[1], base_accuracy, best_fit_accuracy, improvement, random_search_cv.best_estimator_])
        
        
    # Columns we are interested
    columns = ['Model', 'Total_Features', 'Base_Accuracy', 'Best_Fit_Accuracy', 'Improvement', "Best_Estimator"]
    
    # Convert our models summary to dataframe 
    result = pd.DataFrame(model_summary_lst, columns= columns)
    
    return result

## Model Summary for differnt range of features

### Choose all features from data

In [ ]:
all_features_result_df = process_models(X_train, y_train, X_test, y_test)
all_features_result_df.head()

In [ ]:
best_model = all_features_result_df.reset_index().loc[0, 'Best_Estimator']
best_model

### Combine dataframes 

In [ ]:
combined_data = pd.concat([all_features_result_df], ignore_index=True)
combined_data

### Plot graph for combined dataframes with different features range

In [ ]:
fig, ax = plt.subplots(figsize=(15,4))
chart = sns.barplot(x = 'Total_Features', y = 'Best_Fit_Accuracy', hue = 'Model', data = combined_data)
 
# Chart customization
chart.set_title('Accuracy for diffeent models based on features selected')
chart.set(ylim=(0, 2)) # Set max value to more than '1' to see(including legend) the value on graph properly

# Annotate each bar in graph with it's value
for p in chart.patches: 

    display_text = "{0:.4f}".format(p.get_height())
    chart.annotate(display_text, (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')
    